In [ ]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()

In [ ]:
train_input[:10]

In [ ]:
print(train_input.shape, train_target.shape)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 10, figsize=(10,10))
for i in range(10):
    axs[i].imshow(train_input[i], cmap='gray_r')
    axs[i].axis('off')
plt.show()

In [ ]:
items = ["티셔츠 ", "바지", "스웨터", "드레스", "코트",
         "샌달", "셔츠", "스니커즈", "가방", "앵클부츠"]

In [ ]:
[items[i] for i in train_target[:10]]

In [ ]:
import numpy as np

print(np.unique(train_target, return_counts=True))

In [ ]:
train_scaled = train_input / 255
train_scaled = train_scaled.reshape(-1, 28*28)

In [ ]:
train_scaled.shape

In [ ]:
train_input.shape

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier

sc = SGDClassifier(loss="log_loss", max_iter=5, random_state=42)
scores = cross_validate(sc, train_scaled, train_target, n_jobs=-1)
print(np.mean(scores["test_score"]))

In [ ]:
# 인공신경망

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target =\
    train_test_split(train_scaled, train_target, test_size=0.2,
                     random_state=42)

In [ ]:
print(train_scaled.shape)

In [ ]:
print(val_scaled.shape)

In [ ]:
dense = keras.layers.Dense(10, 'softmax', input_shape=(784,))

In [ ]:
model = keras.Sequential(dense)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")

In [ ]:
model.fit(train_scaled, train_target, epochs=10)

In [ ]:
model.evaluate(val_scaled, val_target)

In [ ]:
# 본격적인 딥러닝(2개의 층)

In [ ]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()

In [ ]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255
train_scaled = train_scaled.reshape(-1, 28 * 28)

In [ ]:
train_scaled, val_scaled, train_target, val_target = \
    train_test_split(train_scaled, train_target, 
                     test_size=0.2, random_state=42)

In [ ]:
dense1 = keras.layers.Dense(100, activation="relu", input_shape=(784,))
dense2 = keras.layers.Dense(10, activation="softmax")

In [ ]:
model = keras.Sequential([dense1, dense2])

In [ ]:
model.summary()

In [ ]:
sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss="sparse_categorical_crossentropy",
              metrics="accuracy")
model.fit(train_scaled, train_target, epochs=5)

In [ ]:
model.evaluate(val_scaled, val_target)

In [ ]:
# 07-3 신경망 모델 훈련

In [ ]:
# np.array().reshape(-1, 28 * 28)
# keras.layers.Flatten(input_shape=(28,28))

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255
# train_scaled = train_scaled.reshape(-1, 28 * 28)

train_scaled, val_scaled, train_target, val_target = \
    train_test_split(train_scaled, train_target, 
                     test_size=0.2, random_state=42)

In [ ]:
def model_fn(a_layer=None):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    model.add(keras.layers.Dense(100, activation="relu"))
    if a_layer:
        model.add(a_layer)
    model.add(keras.layers.Dense(10, activation="softmax"))
    return model

In [ ]:
model = model_fn()
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
history = model.fit(train_scaled, train_target, epochs=5, verbose=0)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"])
plt.xlabel("epoch")
plt.ylabel("loss")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["accuracy"])
plt.xlabel("epoch")
plt.ylabel("accuracy")

In [ ]:
model = model_fn()
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics="accuracy")
history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
          validation_data=(val_scaled, val_target))

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(["train", "val"])

In [ ]:
history.history

In [ ]:
# 모델 저장과 복원

In [ ]:
model.save_weights("model-weights.h5")

In [ ]:
model.save("model-whole.h5")

In [ ]:
!ls -al

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))
model.load_weights("model-weights.h5")

In [ ]:
import numpy as np

val_labels = np.argmax(model.predict(val_scaled), axis=-1)
print(np.mean(val_labels == val_target))

In [ ]:
model.predict(val_scaled)[0]

In [ ]:
np.argmax(model.predict(val_scaled)[0])

In [ ]:
# 콜백

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics="accuracy")
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "best-model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=2, restore_best_weights=True)
history = model.fit(train_scaled, train_target, epochs=2000,
          verbose=0, validation_data=(val_scaled, val_target),
          callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
model = keras.models.load_model("best-model.h5")
model.evaluate(val_scaled, val_target)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend(["train", "val"])